# Shaded Area Map
### Based on source from CFBImperialism

In [7]:
# Dependencies

import folium
from folium.plugins import MarkerCluster
# from folium import Fullscreen
from folium import LayerControl
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import json
import sys
import os
from PIL import Image

# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'States_shapefile.shp')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)
# # Transform to the desired CRS if needed
# gdf = gdf.to_crs(epsg=4326)

# Open School Info Table
school_info_path = os.path.join('..', 'data', 'arena_school_info.csv')
school_info = pd.read_csv(school_info_path)

# school_info.head()

#### Create a Dictionary from the school_info dataframe
- dictionary is the default data shape the rest of the code requires

In [8]:


# Make sure hex1, hex2, hex3 are strings
school_info['hex1'] = school_info['hex1'].astype(str)
school_info['hex2'] = school_info['hex2'].astype(str)
school_info['hex3'] = school_info['hex3'].astype(str)

# Transform hex color codes to ensure they are valid - add leading 0s if necessary (6 digits)
def fix_hex_color(hex_color):
    if hex_color.startswith('#'):
        hex_color = hex_color[1:]  # Remove the leading '#'
    hex_color = hex_color.zfill(6)  # Pad with leading zeros to ensure 6 digits
    return f"#{hex_color[-6:]}"  # Return the last 6 characters

# Apply the function to the hex1 column
school_info['hex1'] = school_info['hex1'].apply(fix_hex_color)
# Apply to hex2 & hex3 as well
school_info['hex2'] = school_info['hex2'].apply(fix_hex_color)
school_info['hex3'] = school_info['hex3'].apply(fix_hex_color)

logo_dir = os.path.join('..', 'images', 'logos')


teams = {}
for index, row in school_info.iterrows():
    teams[row['Team']] = {
        'coords': (row['Longitude'], row['Latitude']),
        'color': row['hex1'],
        'logo': os.path.join(logo_dir, f"{row['logo_abv']}.png")
    }

    # Print a few entries to verify
    if index < 5:
        print(f"{row['Team']}: {teams[row['Team']]}")

#### HOTFIX -Changing primary color to hex2 for teams that border other teams with same color
### Change Omaha to hex2
teams['Omaha']['color'] = school_info.loc[school_info['Team'] == 'Omaha', 'hex2'].values[0]
## Arizona State to hex2
teams['Arizona State']['color'] = school_info.loc[school_info['Team'] == 'Arizona State', 'hex2'].values[0]
# print(teams)

# school_info.head()

Air Force: {'coords': (-104.8837269, 39.0137391), 'color': '#003087', 'logo': '..\\images\\logos\\afa.png'}
Alaska: {'coords': (-147.7638406, 64.84212435), 'color': '#236192', 'logo': '..\\images\\logos\\akf.png'}
Alaska Anchorage: {'coords': (-149.8727373, 61.20553644), 'color': '#00583d', 'logo': '..\\images\\logos\\aka.png'}
American Intl: {'coords': (-72.5543263, 42.1180027), 'color': '#000000', 'logo': '..\\images\\logos\\aic.png'}
Arizona State: {'coords': (-111.9108672, 33.4471565), 'color': '#8c1d40', 'logo': '..\\images\\logos\\asu.png'}


### Functions

In [9]:
def get_closest_team(lon, lat, teams):
    min_distance = float('inf')
    closest_team = None
    for team, info in teams.items():
        distance = ((lon - info['coords'][0]) ** 2 + (lat - info['coords'][1]) ** 2) ** 0.5
        if distance < min_distance:
            min_distance = distance
            closest_team = team
    return closest_team

# Add a new column to the GeoDataFrame for the closest team
gdf['closest_team'] = gdf.geometry.apply(lambda x: get_closest_team(x.centroid.x, x.centroid.y, teams))
gdf['color'] = gdf['closest_team'].apply(lambda x: teams[x]['color'])


# Create a Folium map centered around Michigan
# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://stamen-tiles.a.ssl.fastly.net/watercolor/{z}/{x}/{y}.jpg',
#                attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors')

# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://stamen-tiles.a.ssl.fastly.net/terrain/{z}/{x}/{y}.jpg', 
#                attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors')

# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
#                attr='© OpenStreetMap contributors © CARTO')

# 
m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', 
               attr='© OpenStreetMap contributors © CARTO')
# m = folium.Map(location=[44.5, -85.5], zoom_start=6, 
#                 tiles='Stamen Watercolor',
#                 attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors')

# Add states to the map with a darker outline
for _, row in gdf_states.iterrows():  # Assuming gdf_states contains the state geometries
    geo_json = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,  # Ensure it's in the correct format
        'properties': {}
    }

    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': 'none',  # No fill for states
            'color': 'darkblue',  # Darker color for state borders
            'weight': 2.5  # Thicker outline for states
        }
    ).add_to(m)

# Add counties to the map
for _, row in gdf.iterrows():
    geo_json = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,  # Ensure it's in the correct format
        'properties': {
            'closest_team': row['closest_team'],  # Add properties like 'closest_team'
            'color': row['color']
        }
    }

    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': feature['properties']['color'],  # Access color from properties
            'color': 'black',
            'weight': 0.4,
            'fillOpacity': 0.6
        },
        highlight_function=lambda x: {'weight': 3, 'color': 'blue'},  # Highlight on hover
        tooltip=folium.GeoJsonTooltip(fields=['closest_team'], aliases=['Closest Team:'])
    ).add_to(m)




for team, info in teams.items():
    # Example of scaling icon size based on a zone factor
    zone_factor = info.get('zone_factor', 1)  # Default to 1 if no zone factor exists
    icon_size = int(50 * zone_factor)  # Adjust 50 to the base size you want
    
    folium.Marker(
        location=[info['coords'][1], info['coords'][0]],
        icon=folium.CustomIcon(info['logo'], icon_size=(icon_size, icon_size))  # Scaled size
    ).add_to(m)

## Other Add-ons and Options
# folium.plugins.Fullscreen().add_to(m) # add a fullscreen toggle
# folium.LayerControl().add_to(m) # Add layer control - allow user to switch base maps

# Save the map to an HTML file
output_file = os.path.join('..', 'TEMP', 'school_map_test7.html')
m.save(output_file)


    